# Проект Исследование надежности заемщиков
## Получение данных

In [1]:
import pandas as pd

Общая информация о данных.

In [2]:
data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

Выведим основную информацию о датафрейме с помощью метода info().

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Выведите первые 10 строчек датафрейма data 

In [4]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


## Предобработка данных
### Пропущенные значения
Выведим количество пропущенных значений для каждого столбца.

In [5]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В двух столбцах есть пропущенные значения. Столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`.

In [6]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

Заполните пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.

In [7]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [8]:
data.isna().sum() #Проверка

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Обработка аномальных значений

В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Обработаем значения в этом столбце: заменим все отрицательные значения положительными с помощью метода `abs()`.

In [9]:
data['days_employed'] = data['days_employed'].abs()

Для каждого типа занятости выведим медианное значение трудового стажа `days_employed` в днях.

In [10]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставьте их как есть.

Выведим перечень уникальных значений столбца `children`.

In [11]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

В столбце children есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма data.

In [12]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [13]:
data['children'].unique() #Проверка

array([1, 0, 3, 2, 4, 5])

### Изменение типов данных

Заменим вещественный тип данных в столбце total_income на целочисленный с помощью метода astype().

In [14]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

Обработаем неявные дубликаты в столбце education. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к нижнему регистру.

In [15]:
data['education'] = data['education'].str.lower()

In [16]:
data.duplicated().sum() # количество явных дубликатов

71

In [17]:
data = data.drop_duplicates()

## Категоризация данных

На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


Используем собственную функцию с именем `categorize_income()` и метод `apply()`.

In [18]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [19]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

Выведим на экран перечень уникальных целей взятия кредита из столбца purpose

In [20]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.


Используем собственную функцию с именем `categorize_purpose()` и метод `apply()`.

In [21]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [22]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

## Исследование данных
### Есть ли зависимость между количеством детей и возвратом кредита в срок?
Для начала сформулируем гипотезу: **"Чем больше детей у заемщика, тем вероятнее он вернет в срок кредит"**. <br>
Проверять гипотезу будем на таком показателе как доля просроченной задолженности: отношение просроченных кредитов ко всем. <br> 

In [23]:
data.pivot_table(index = 'children', values = 'debt', aggfunc = ['count', 'sum', 'mean'])\
    .sort_values(by = ('mean', 'debt'), ascending = False)\
    .style.format({('mean', 'debt') : '{:.2%}'})

,count,sum,mean
,debt,debt,debt
children,,,
4,41,4,9.76%
2,2052,194,9.45%
1,4808,444,9.23%
3,330,27,8.18%
0,14091,1063,7.54%
5,9,0,0.00%


**Промежуточный вывод:** <br>
Данные опровергают гипотезу о том, что чем больше у заемщика детей, тем более вероятно он вернет кредит в срок. <br>
Доля просрочки у заемщиков без детей ниже, чем у заемщиков с одним, двумя, тремя и четыремя детьми.
При этом, например, доля кредитов с нарушением срока платежа у заемщиков с четыремя детьми самая высокая среди рассматриваемых групп.<br>
При этом нельзя сказать, что чем больше детей у заемщика, тем хуже у таких заемщиков с кредитной дисциплиной. 
Единственной группой заемщиков не имеющей нарушений в уплате кредитов стала весьма малочисленная группа заемщиков с пятью детьми.<br>
Отвечая на вопрос, для целей скоринговой модели оценки кредитоспособности, очевидно, что наиболее надежными заемщиками будут люди с пятью детьми(но таких заемщиков очень мало) и люди без детей вовсе. 

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

Аналогично расчитаем показатель просроченной задолженности для групп заемщиков в зависимости от семейного положения. <br> Гипотезу сформулируем как: **"Возврат кредита в срок более вероятен среди заемщиков состоящих в браке ( как в официальном так и в гражданском), чем среди не состоящих в браке заемщиков(не замужние/не женатые, разведенные)"** <br>
В таблице представлены следующие данные по группам в зависимости от семейного положения: количество выданных кредитов, количество кредитов с нарушением срока платежа, доля просроченных кредитов в каждой группе.

In [24]:
data.groupby(by = 'family_status')['debt'].agg(['count', 'sum', 'mean'])

,count,sum,mean
family_status,,,
Не женат / не замужем,2796,273,0.097639
в разводе,1189,84,0.070648
вдовец / вдова,951,63,0.066246
гражданский брак,4134,385,0.093130
женат / замужем,12261,927,0.075606


**Промежуточный вывод:** <br>
Данные говорят о следующем: меньше всего доля просроченных кредитов среди вдов и вдовцов. Заемщики в разводе допускают просрочку по кредиту реже, чем состоящие в браке заемщики. Хуже перечисленных ранее категорий дела со своевременным возвратом кредита обстоят у заемщиков состоящих в гражданском браке, но даже они допускают нарушение условий кредитного договора реже, чем не женатые / не замужние заемщики. <br>
Таким образом, гипотеза частично подтвердилась. Для скоринговой системы наименее кредитоспособные заемщики - это не женатые / не замужние, наиболее надежные - овдовевшие и разведенные заемщики.

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Расчитаем показатель просроченной задолженности для групп заемщиков в зависимости от уровня дохода.
Гипотезу сформулируем как: **"Чем выше уровень дохода, тем более вероятно, что заемщик вернет кредит в срок"** <br>
В таблице представлены следующие данные по группам в зависимости от категории дохода: количество выданных кредитов, количество кредитов с нарушением срока платежа, доля просроченных кредитов в каждой группе.

In [25]:
data.pivot_table(index = 'total_income_category', values = 'debt', aggfunc = ['count', 'sum', 'mean'])\
    .sort_values(by = ('count', 'debt'), ascending = False)\
    .style.format({('mean', 'debt') : '{:.2%}'})

,count,sum,mean
,debt,debt,debt
total_income_category,,,
C,15921,1353,8.50%
B,5014,354,7.06%
D,349,21,6.02%
A,25,2,8.00%
E,22,2,9.09%


**Промежуточный вывод:** <br>
Гипотеза не подтверждается. В группе с наименьшим зарабатком, очевидно наибольшая доля просрочки, хотя важно отметить что одновременно это и самая малочисленная группа. Наименьшая доля просроченных кредитов в категории Д, а во всех вышестоящих категориях с большим доходом, доля выше, чем в Д. <br>
Кроме того, можно отметить, что даже самая обеспеченная группа заемщиков допускает просрочки по кредитному договору. <br> Для скоринговой модели наибольшую оценку следует присвоить заемщику с доходом от 30 тыс до 50 тыс(Д) или от 200 тыс до 1млн(Б).


### Как разные цели кредита влияют на его возврат в срок?

Расчитаем показатель просроченной задолженности  в зависимости от цели кредита. <br>
<br> В таблице представлены следующие данные по группам в зависимости от цели кредита: количество выданных кредитов, количество кредитов с нарушением срока платежа, доля просроченных кредитов в каждой группе.

In [26]:
data.groupby(by = 'purpose_category')['debt'].agg(['count', 'sum', 'mean'])

,count,sum,mean
purpose_category,,,
операции с автомобилем,4279,400,0.093480
операции с недвижимостью,10751,780,0.072551
получение образования,3988,369,0.092528
проведение свадьбы,2313,183,0.079118


**Промежуточный вывод:** <br>
Меньше всего доля просроченной задолженности в категории "операции с недвижимостью". Немного менее вероятен возврат в срок кредит взятый на свадьбу. Небольшая разница между долей просроченных автокредитов и образовательных кредитов. Эти цели кредита в скоринговой оценке получили бы низший балл.

## Итоговые выводы
В результате проделанной работы мы обработали пропуски в таблице, категоризировали ее и построили сводные таблицы для получения ответов на поставленные вопросы.

Были получены необходимые для создания скоринговой системы ответы на следующие вопросы: <br>
1. Есть ли зависимость между количеством детей и возвратом кредита в срок? <br>
Доля просрочки у заемщиков без детей ниже, чем у заемщиков с одним, двумя, тремя и четыремя детьми. Согласно данным, наиболее добросовестные заемщики имеют пятерых детей или не имеют вовсе. 
2. Есть ли зависимость между семейным положением и возвратом кредита в срок? <br>
Анализ данных показал, что наименее кредитоспособные заемщики - это не женатые / не замужние, наиболее надежные - овдовевшие и разведенные заемщики.
3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок? <br>
Наибольшая вероятность вернуть выданный кредит в срок у заемщиков с доходом от 30 тыс до 50 тыс(Д) или от 200 тыс до 1млн(Б). Наиболее и наименее обеспеченные клиенты банка допускают просрочку чаще, чем клиенты отнесенные к категории Д и Б.
4. Как разные цели кредита влияют на его возврат в срок? <br>
Меньше всего доля просроченной задолженности в категории "операции с недвижимостью". Немного менее вероятен возврат в срок кредит взятый на свадьбу. Небольшая разница между долей просроченных автокредитов и образовательных кредитов. 